In [32]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [26]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [4]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [5]:
#! Set these variables
work_dir_name = "choroid_pineal_pituitary2"

In [6]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [12]:
labels = ["choroid_t1_flair", "pineal", "pituitary"]
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(["choroid_t1_flair", "pineal", "pituitary"], suffix_list=["CH", "SRS", "ED", "DT"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-12-02 14:41:32.019 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/5 [00:00<?, ?it/s]2024-12-02 14:41:32.255 | INFO     | mri_data.utils:combine_labels:84 - Combining labels for Scan(subid=1188, sesid=20200720)
2024-12-02 14:41:32.277 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms1188/ses-20200720/choroid_t1_flair-ED.pineal-CH.pituitary-CH.nii.gz
 20%|██        | 1/5 [00:00<00:01,  3.89it/s]2024-12-02 14:41:32.334 | INFO     | mri_data.utils:combine_labels:84 - Combining labels for Scan(subid=2081, sesid=20170204)
2024-12-02 14:41:32.358 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpioneer_bids/sub-ms2081/ses-20170204/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz
2024-12-02 14:41:32.420 | INFO     | mri_data.utils:combine_labels:84 - Combining labels for Scan(subid=1259, sesid=20200803)
2024-12-02 14:41:32.429 | SUCCESS  | monai_training.preprocess:log:22 - Saved /mnt/h/3Tpione

Calculate Dice Score for choroid

In [ ]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.root / "choroid_t1_flair.nii.gz").get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=1)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1109: 0.79
1188: 0.80
1259: 0.74
1547: 0.78
2081: 0.80
----------
Mean: 0.78
Std: 0.02


Calculate IoU for choroid

In [48]:
iou = []
print("IoU")
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.root / "choroid_t1_flair.nii.gz").get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.iou(seg1, seg2, seg2_val=1)
    iou.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(iou) / len(iou)))
print("Std: {:0.2f}".format(statistics.stdev(iou)))

print(iou)

IoU
1109: 0.65
1188: 0.66
1259: 0.59
1547: 0.64
2081: 0.67
----------
Mean: 0.64
Std: 0.03
[0.6500157084511468, 0.6618705035971223, 0.592372288313506, 0.6447275300778486, 0.6728179551122194]


Calculate Hausdorff Distance for Choroid

In [49]:
haus = []
print("Hausdorff Dist")
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.root / "choroid_t1_flair.nii.gz").get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

Hausdorff Dist
1109: 47.03
1188: 4.12
1259: 20.59
1547: 13.27
2081: 94.88
----------
Mean: 35.98
Std: 36.60
[47.031904065219386, 4.123105625617661, 20.591260281974, 13.2664991614216, 94.87887014504336]


Calculate Dice Score for pineal

In [53]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pineal", ["CH", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=2)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1109: 0.84
1188: 0.86
1259: 0.88
1547: 0.83
2081: 0.89
----------
Mean: 0.86
Std: 0.03
[0.8352059925093633, 0.8642857142857143, 0.8839427662957074, 0.8253706754530478, 0.885478158205431]


Calculate IoU for pineal

In [54]:
iou = []
print("IoU")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pineal", ["CH", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.iou(seg1, seg2, seg2_val=2)
    iou.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(iou) / len(iou)))
print("Std: {:0.2f}".format(statistics.stdev(iou)))

print(iou)

IoU
1109: 0.72
1188: 0.76
1259: 0.79
1547: 0.70
2081: 0.79
----------
Mean: 0.75
Std: 0.04
[0.7170418006430869, 0.7610062893081762, 0.792022792022792, 0.7026647966339411, 0.7944915254237288]


Calculate Hausdorff Dist for pineal

In [55]:
haus = []
print("haus")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pineal", ["CH", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=2)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

haus
1109: 2.45
1188: 1.41
1259: 1.41
1547: 5.48
2081: 1.00
----------
Mean: 2.35
Std: 1.83
[2.449489742783178, 1.4142135623730951, 1.4142135623730951, 5.477225575051661, 1.0]


Calculate DICE score for pituitary

In [50]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pituitary", ["CH", "ED", "DT"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=3)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1109: 0.78
1188: 0.89
1259: 0.89
1547: 0.83
2081: 0.81
----------
Mean: 0.84
Std: 0.05
[0.7785419532324622, 0.8917337234820776, 0.8895244905255631, 0.8349575605155611, 0.8089430894308943]


Calculate IoU for pituitary

In [51]:
iou = []
print("IoU")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pituitary", ["CH", "ED", "DT"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.iou(seg1, seg2, seg2_val=3)
    iou.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(iou) / len(iou)))
print("Std: {:0.2f}".format(statistics.stdev(iou)))

print(iou)

IoU
1109: 0.64
1188: 0.80
1259: 0.80
1547: 0.72
2081: 0.68
----------
Mean: 0.73
Std: 0.07
[0.6373873873873874, 0.8046204620462046, 0.8010302640051513, 0.7166756610901241, 0.6791808873720137]


Calculate Hausdorff Dist for pituitary

In [52]:
haus = []
print("haus")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pituitary", ["CH", "ED", "DT"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=3)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

haus
1109: 3.61
1188: 3.00
1259: 2.24
1547: 3.32
2081: 3.16
----------
Mean: 3.06
Std: 0.51
[3.605551275463989, 3.0, 2.23606797749979, 3.3166247903554, 3.1622776601683795]


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [ ]:
inference: Scan = ensemble_out_dataset[0]
label_path = inference.label_path
new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
utils.set_label_value(label_path, new_label_path, 2)

PosixPath('/home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble_val2.nii.gz')

In [10]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, "-s", manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1033/ses-20171117/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1152/ses-20170529/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_resegment1/ensemble_output/sub-ms1152/ses-20170529/flair.t1_ensemble.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1163/ses-20180907/t1.nii.gz -s /home/s

Produce commands to open images and labels in freeview

In [1]:
image_paths = ["one", "twp", "three"]
step1 = " -g ".join(image_paths)
step1.split(" ")

['one', '-g', 'twp', '-g', 'three']

In [ ]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat